---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [159]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [160]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [161]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data = load_university_towns()
    data = clean_state_names(data)
    towns = clean_university_towns(data)  
    return towns
    
def load_university_towns():
    with open('university_towns.txt') as f:
        data = np.array([x.split(" (") for x in f.readlines()])
    return data

def clean_state_names(data):
    for index, row in enumerate(data):
        if len(row) == 1 and 'edit' in row[0]:
            data[index][0] = data[index][0].split('[')[0]
    return data
    
def clean_university_towns(data):
    towns = pd.DataFrame(columns=['State','RegionName'])    
    for item in data:
        if '\n' not in item[-1]:
            state = item[0]
        if state != item[0]:
            towns = towns.append({'State': state, 'RegionName': item[0]}, ignore_index=True)
    return towns

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


The following two functions load the gdp data and create a new column flagging when there is a recession, with 1 being no recession and 0 being a recession.

In [215]:
def load_gdp_data():
    return pd.read_excel('gdplev.xls', usecols='E,G', names=['Quarter', 'GDP'], skiprows=220,header=None)

def identify_gdp_change(data):    
    change = np.array(data.GDP - data.GDP.shift(1))
    recessions = np.empty(0, dtype='int')
    prior = 0
    for index, quarter in enumerate(change):
        if quarter < 0:
            next = max(-1, prior - 1)
        else:
            next = min(1, prior + 1)
        recessions = np.append(recessions, next)
        prior = next
    return change, recessions

quarterly_gdp = load_gdp_data()
quarterly_gdp['Change'], quarterly_gdp['Recession'] = identify_gdp_change(quarterly_gdp)
pd.set_option('display.max_rows', 15)
print(quarterly_gdp)

   Quarter      GDP  Change  Recession
0   2000q1  12359.1     NaN          1
1   2000q2  12592.5   233.4          1
2   2000q3  12607.7    15.2          1
3   2000q4  12679.3    71.6          1
4   2001q1  12643.3   -36.0          0
..     ...      ...     ...        ...
61  2015q2  16374.2   105.2          1
62  2015q3  16454.9    80.7          1
63  2015q4  16490.7    35.8          1
64  2016q1  16525.0    34.3          1
65  2016q2  16583.1    58.1          1

[66 rows x 4 columns]


In [226]:
def get_recession_start(quarterly_gdp):
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return quarterly_gdp['Quarter'][quarterly_gdp.index[quarterly_gdp['Recession'] == -1][0] - 1]
    
starting_quarter = get_recession_start(quarterly_gdp)
print(starting_quarter)

2008q3


In [227]:
def get_recession_end(quarterly_gdp,start):
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    starting_index = get_index(quarterly_gdp,start)
    growth_quarters = quarterly_gdp.index[quarterly_gdp['Recession'] == 1].tolist()
    
    return quarterly_gdp['Quarter'][[i for i in growth_quarters if i > starting_index][0]]

def get_index(quarterly_gdp, quarter):
    return quarterly_gdp.index[quarterly_gdp['Quarter'] == quarter].tolist()[0]

ending_quarter = get_recession_end(quarterly_gdp,'2008q3')
print(ending_quarter)

2009q4


In [237]:
def get_recession_bottom(quarterly_gdp,start,end):
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    starting_index = get_index(quarterly_gdp, start)
    ending_index = get_index(quarterly_gdp, end) + 1
    
    recession = quarterly_gdp.iloc[starting_index:ending_index]
    
    return recession[recession['GDP'] == min(recession['GDP'])]['Quarter'].values[0]
    
recession_bottom = get_recession_bottom(quarterly_gdp,starting_quarter,ending_quarter)
print(recession_bottom)

2009q2


In [278]:
def convert_housing_data_to_quarters(states):
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = load_housing_data()
    return clean_housing_data(housing_data, states)

def load_housing_data():
    return pd.read_csv('City_Zhvi_AllHomes.csv')

def clean_housing_data(housing_data, states):
    
    # Create list of columns for summarized housing data by quarter
    quarters = [str(year)+'q'+str(quarter//3) for year in range(2000,2017) for quarter in [3,6,9,12]]
    quarters.pop()
    
    # Identify the columns to average for each quarter in the summarized housing data
    quarter_dict = {'q1': ['-01','-02','-03'],
                    'q2': ['-04','-05','-06'],
                    'q3': ['-07','-08','-09'],
                    'q4': ['-10','-11','-12']}
    qcol = {quarter: [quarter[0:4] + suffix for suffix in quarter_dict[quarter[4:6]]] for quarter in quarters}
    qcol['2016q3'].pop()
    
    # Create summarized housing data dataframe with State and RegionName from original housing data
    summarized_housing_data = housing_data[['State','RegionName']].copy()
    
    # Populate average quarterly data into the summarized housing data dataframe
    for quarter in quarters:
        summarized_housing_data[quarter] = housing_data[qcol[quarter]].mean(axis=1)
    
    # Create appropriate index
    summarized_housing_data['State'].replace(states, inplace=True)
    summarized_housing_data.set_index(['State','RegionName'], inplace=True)
    
    return summarized_housing_data
    
    
convert_housing_data_to_quarters(states)

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"